In [1]:
import pandas as pd
import numpy as np

In [2]:
nba = pd.read_csv('https://raw.githubusercontent.com/jkropko/contrans/main/examples/ASA%20All%20NBA%20Raw%20Data.csv')

In [3]:
nba.columns

Index(['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct',
       'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate',
       'Opponent_off_rtg', 'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm',
       'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP',
       'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_s

In [4]:
nba = nba[['game_id', 'game_date', 'OT', 'H_A', 'Team_Abbrev', 'Team_Score',
       'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct',
       'Team_ft_rate', 'Team_off_rtg', 'Inactives', 'Opponent_Abbrev',
       'player', 'player_id', 'starter', 'mp', 'fg', 'fga',
       'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb',
       'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus',
       'did_not_play', 'is_inactive', 'off_rtg', 'def_rtg', 'bpm']]

In [5]:
pd.set_option('display.max_rows',42)
pd.set_option('display.max_columns',42)
nba.head(3).T
#Transpose since it's easier to read

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


### Making a Relational Database

#### First Normal Form

3 Rules (Don't do more than what the rules are saying):

   1. Everytable must have a primary key? Yes, player_id and game_id together.
    
   2. The values inside every cell in every table must be atomic? The Inactives attribute 

In [6]:
nba = nba.drop(['Inactives'],axis=1)
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


    3. There are no repeating groups

#### Second Normal Form

Every non-prime columns must depend on the ENTIRE primary key (gameid + playerid) and not just part of the primary key (just gameid or playerid).

We have this in this entity since OT and H_A are not dependent on the player 

If you primary key is just one column, then you can violate this rule. Then, let's do a second player id column! Replace many columns with just one column each.

In [11]:
nba['game_player_id'] = nba['game_id']+'_'+nba['player_id']
nba.head(3).T

,0,1,2
game_id,202204100BRK,202204100BRK,202204100BRK
game_date,2022-04-10,2022-04-10,2022-04-10
OT,0,0,0
H_A,A,A,A
Team_Abbrev,IND,IND,IND
Team_Score,126,126,126
Team_pace,103.9,103.9,103.9
Team_efg_pct,0.543,0.543,0.543
Team_tov_pct,5.9,5.9,5.9
Team_orb_pct,20.8,20.8,20.8


1. The data must meet all the criteria to be 1NF,

2. Wikipedia lists the second crition for 2NF as:

    It does not have any non-prime attribute that is functionally dependent on any proper subset of any candidate key of the relation. A non-prime attribute of a relation is an attribute that is not a part of any candidate key of the relation.

#### Third normal form

1. All of the criteria necessary for the database to be in 2NF.

2. “Every non-prime attribute … is non-transitively dependent on every [attribute]”

This just means no transitive dependancy. If there is a functional dependency, you have to get rid of it by creating another table.

Calculated column:

In [16]:
nba = nba.drop(['fg_pct'],axis=1)
nba = nba.drop(['fg3_pct'],axis=1)
nba = nba.drop(['ft_pct'],axis=1)
nba.head(3).T

KeyError: "['fg_pct'] not found in axis"

Transitive dependencies:

- Some colunmns depend on player
- Some columns depend on game
- game columns depend on team